In [ ]:
!python -m pip install git+https://github.com/huggingface/transformers -q

In [ ]:
from transformers import Qwen2VLForConditionalGeneration, AutoTokenizer, AutoProcessor

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
from PIL import Image
from glob import glob
import requests
import re
from datetime import datetime

In [ ]:
import torch,gc
torch.cuda.empty_cache()
gc.collect()

In [ ]:
def combine(a,b):
    im1 = cv2.imread(a)
    im2 = cv2.imread(b)
    im1 = cv2.resize(im1,(224,224))
    im2 = cv2.resize(im2,(224,224))
    c2 = np.hstack((im1,im2))
    return c2

In [ ]:
model = Qwen2VLForConditionalGeneration.from_pretrained( "Qwen/Qwen2-VL-2B-Instruct", torch_dtype="auto", device_map="auto", )
processor = AutoProcessor.from_pretrained( "Qwen/Qwen2-VL-2B-Instruct" )

In [ ]:
gg = glob('/kaggle/input/abcdef/*.jpg')
len(gg)

In [ ]:
def rotate_image(src):
    r0 = cv2.imread(src)
    r1 = cv2.rotate(r0, cv2.ROTATE_90_CLOCKWISE)
    r2 = cv2.rotate(r0, cv2.ROTATE_180)
    r3 = cv2.rotate(r0, cv2.ROTATE_90_COUNTERCLOCKWISE)
    return r0,r1,r2,r3

In [ ]:
def plot_images(r1,r2,r3,r4):
    # Plotting the images
    plt.figure(figsize=(20, 5))
    # First image (r1)
    plt.subplot(1, 4, 1)
    plt.imshow(r1)
    plt.title("Rotated Image 1")
    # Second image (r2)
    plt.subplot(1, 4, 2)
    plt.imshow(r2)
    plt.title("Rotated Image 2")
    # Third image (r3)
    plt.subplot(1, 4, 3)
    plt.imshow(r3)
    plt.title("Rotated Image 3")
    # Fourth image (r4)
    plt.subplot(1, 4, 4)
    plt.imshow(r4)
    plt.title("Rotated Image 4")
    # Display the images
    plt.show()

In [ ]:
def is_valid_date(date_str):
    # List of known date formats
    date_formats = [
        "%d/%m/%Y", "%d-%m-%Y", "%Y/%m/%d", "%Y-%m-%d", 
        "%m/%d/%Y", "%m-%d-%Y", "%d %b %Y", "%d %B %Y", 
        "%d/%m/%y", "%d-%m-%y", "%Y%m%d"
    ]
    # Try parsing the date using each format
    for date_format in date_formats:
        try:
            # Try to parse the date string using the current format
            datetime.strptime(date_str, date_format)
            return True
        except ValueError:
            continue
    return False

In [ ]:
gg = ['/kaggle/input/abcdef/c2-2.jpg']

In [ ]:
def final_model(gg):
    oc = 0
    for i in gg:       
        r1,r2,r3,r4 = rotate_image(i)
        image1 = Image.fromarray(r1)
        image2 = Image.fromarray(r2)
        image3 = Image.fromarray(r3)
        image4 = Image.fromarray(r4)
        images = [image1, image2, image3, image4]
        messages = [{ "role": "user",
                      "content": [
                    { "type": "image",},
                    { "type": "text", "text": {"find the class and expiry date of the product.Output it in the following format: class: class of the product, expiry date: expiry date of the product. Note: Don't print any extra output.Also, the classes shoud be from any one of these only [Staples, Snacks&Beverages, Packaged Food, Personal&Babycare, HouseholdCare, Dairy&eggs, Home&Kitchen]"}
                    }] }]
        outputs=[]
        for img in images:
            text_prompt = processor.apply_chat_template(messages, add_generation_prompt=True)
            inputs = processor( text = [text_prompt], images = [img], padding = True, return_tensors = "pt" )
            inputs = inputs.to("cuda")
            output_ids = model.generate(**inputs, max_new_tokens=1024)
            generated_ids = [output_ids[len(input_ids) :]
            for input_ids, output_ids in zip(inputs.input_ids, output_ids)]
            output_text = processor.batch_decode( generated_ids, skip_special_tokens=True, clean_up_tokenization_spaces=True )
            outputs.append(output_text[0])
        final_dict = {
            'class' : set(),
            'expiry_date' : set()
        }
        for input_str in outputs:
            classes = re.findall(r'class: ([^,]+)', input_str)
            expiry_dates = re.findall(r'expiry date: ([^,]+)', input_str)
            valid_expiry_dates = [date for date in expiry_dates if is_valid_date(date)]
            if len(classes)>0:
                final_dict['class'].add(classes[0])
            if len(valid_expiry_dates)>0:
                final_dict['expiry_date'].add(valid_expiry_dates[0])
        print(final_dict)
        plot_images(image1, image2, image3, image4)
        oc += 1
        if oc==1:
            break

In [ ]:
final_model(gg)